##### Copyright 2018 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a Deep Q Network with TF-Agents

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/1_dqn_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Introduction


This example shows how to train a [DQN (Deep Q Networks)](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf)  agent on the Cartpole environment using the TF-Agents library.

![Cartpole environment](https://raw.githubusercontent.com/tensorflow/agents/master/docs/tutorials/images/cartpole.png)

It will walk you through all the components in a Reinforcement Learning (RL) pipeline for training, evaluation and data collection.


To run this code live, click the 'Run in Google Colab' link above.


## Setup

If you haven't installed the following dependencies, run:

In [ ]:
!sudo apt-get install -y xvfb ffmpeg
!pip install gym
!pip install 'imageio==2.4.0'
!pip install PILLOW
!pip install pyglet
!pip install pyvirtualdisplay
!pip install tf-agents

In [ ]:
import random
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod

class TicTacToe():

    def __init__(self):
        self.state = '---------'

    # Resets the game
    def reset_game(self):
        self.state = '---------'

    # Check if game over
    def game_over(self):

        # Check if there are any possible moves
        if '-' in self.state:
            return None

        # Each list corresponds to the values to check to see if a winner is there
        checks = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [0, 3, 6], [1, 4, 7] , [2, 5, 8], [0, 4, 8], [2, 4, 6]]
        for check in checks:
            # Check to see if the strings have a winner
            test = self.state[check[0]] + self.state[check[1]] + self.state[check[2]]
            if test == 'XXX':
                return 'X'
            elif test == 'OOO':
                return 'O'

        # Return tie in the last case
        return 'T'

    # Does a move given an action (number) and a piece
    def play_move(self, move, ID):

        # Return false if illegal move
        if self.state[move] != '-':
            return False

        # Plays a move depending on what number is fed in
        self.state = self.state[:move] + ID + self.state[move+1:]

        # Return true if legal move
        return True

    # Gets the reward via ID
    def get_reward(self, result, ID):

        # No reward for a non-game ending state
        if result == 'T':
            return 0.5
        elif result == ID:
            return 1
        else:
            return 0

    # Trains a bot against a random player
    def train(self, a_1, a_2, batch):

        # Exit if the same ID
        if a_1.ID == a_2.ID:
            return

        # Store total wins
        a_1_wins = 0
        a_1_ties = 0

        # Train on 1000 games per session
        for i in range(batch):

            # Store the result
            result = None

            # Make a turn variable to see who's it is
            turn = 'X'

            # Each game is at most 9 turns
            for i in range(9):

                # Play moves
                if a_1.ID == turn:

                    # Get move and play move
                    move = a_1.get_move(self.state)
                    self.play_move(move, a_1.ID)

                    # Flip turn
                    turn = 'O'

                    # Check game over
                    result = self.game_over()
                    if (result != None):
                        break

                else:
                    # Do random move for other player
                    move = a_2.get_move(self.state)
                    self.play_move(move, a_2.ID)

                    # Flip turn
                    turn = 'X'

                    # Check game over
                    result = self.game_over()
                    if (result != None):
                        break

            # Update if game over
            if result != None:
                a_1.calculate(self.get_reward(result, a_1.ID))
                a_2.calculate(self.get_reward(result, a_2.ID))
                a_1.reset()
                a_2.reset()
                self.reset_game()

                # Add one to wins if won
                if result == a_1.ID:
                    a_1_wins += 1
                elif result == 'T':
                    a_1_ties += 1

        return (a_1_wins, a_1_ties)


# Superclass for aribtrary agent
class Agent:

    def __init__(self, ID, name):
        self.ID = ID
        self.name = name

    # Called to get the next move
    @abstractmethod
    def get_move(self, state):
        pass

    # Called at the end of each game to calculate parameters
    @abstractmethod
    def calculate(self, result):
        pass

    # Called to reset the agent
    @abstractmethod
    def reset(self):
        pass

class Random(Agent):

    def __init__(self, ID):
        # Call superconstructor
        super().__init__(ID, 'Random Agent')

    def get_move(self, state):
        return random.choice([i for i in range(9) if state[i] == '-'])

    def calculate(self, result):
        return

    def reset(self):
        return

class Neural_Q_Agent(Agent):

    def __init__(self, ID):
        # Call superconstructor
        super().__init__(ID, 'Neural Q Agent')

    def get_move(self, state):
        return random.choice([i for i in range(9) if state[i] == '-'])

    def calculate(self, result):
        return

    def reset(self):
        return

class Tabular_Q_Agent(Agent):

    def __init__(self, ID, random=False):

        # Call superconstructor
        super().__init__(ID, 'Tabular Q Agent')

        # State sets, stored as dictionary
        self.states = {}
        self.alpha = 0.9
        self.discount = 0.96
        self.init = 0.6
        self.recorded_states = []

        # For gathering data
        self.gather_data = False
        self.training = []
        self.labels = []

    # For a new game
    def reset(self):
        self.recorded_states = []

    # Calculate stuff for training
    def neural_net_calc(self, result):

        self.recorded_states.reverse()

        # Flag for the first time
        first = True
        maximum = 0

        # For the first case, we just use the reward
        for state in self.recorded_states:

            # Test these three characters
            text = ['X', 'O', '-']
            res = []
            for char in text:
                if state[i] == char:
                    res.training.append(1)
                else:
                    res.training.append(0)

            # Add the list to the training data
            self.training.append(res)

            # Do special update if first
            if first:
                # Set the win condition to 1
                self.states[state[0]][state[1]] = reward
                first = False
            else:
                if self.gather_data:

                self.states[state[0]][state[1]] = (1 - self.alpha) * self.states[state[0]][state[1]] + self.alpha * self.discount * maximum

            # Store the previous max
            maximum = max(self.states[state[0]])


    # Updates the policy
    def calculate(self, reward):

        # Skip this if we're just gathering data
        if self.gather_data:
            return

        # Iterate through the states
        self.recorded_states.reverse()

        # Flag for the first time
        first = True
        maximum = 0

        # For the first case, we just use the reward
        for state in self.recorded_states:

            # Do special update if first
            if first:
                # Set the win condition to 1
                self.states[state[0]][state[1]] = reward
                first = False
            else:
                if self.gather_data:

                self.states[state[0]][state[1]] = (1 - self.alpha) * self.states[state[0]][state[1]] + self.alpha * \
                                                  self.discount * maximum

            # Store the previous max
            maximum = max(self.states[state[0]])

    # Get q-value list for a key with lazy initialization
    def get_q(self, state):

        # If the key is there return it
        if state in self.states:
            return self.states[state]
        # Otherwise initialize it and set illegal moves to -Inf
        else:
            res = []
            # Initialize the array with -inf for values that are illegal
            for i in range(9):
                if state[i] == '-':
                    res.append(self.init)
                else:
                    res.append(float('-inf'))

            self.states[state] = res
            return res

    # Generates possible states to move to
    def generate_moves(self, state, piece):

        # Create a list to store
        moves = []

        # Iterate through and add 'X' or 'O' wherever possible
        for i in range(9):
            if state[i] == '-':
                moves.append(state[:i] + piece + state[i+1:])

        return moves

    # Get an optimal move
    def get_move(self, state):


        # Get the q-values for this
        q_vals = self.get_q(state)

        # Get location of all max values and select a random one
        max_q = max(q_vals)
        move = random.choice([i for i, j in enumerate(q_vals) if j == max_q])

        # Record a tuple of board state and index of move
        self.recorded_states.append((state, move))

        return move


# Initialize environment
p1 = Tabular_Q_Agent('X')
p2 = Tabular_Q_Agent('O')
t = TicTacToe()

# Store results
wins = []
ties = []
losses = []
count = []
batch = 100
for i in range (500):
    print(i)
    res = t.train(p1, p2, batch)
    wins.append(res[0] / batch)
    ties.append(res[1] / batch)
    losses.append((batch - res[0] - res[1]) / batch)
    count.append(batch * i)


# Print stuff for matplotlib
plt.ylabel('Outcomes (%)')
plt.xlabel('Number of Games')
plt.title('P1: ({}) vs. P2: ({})'.format(p1.name, p2.name))

plt.plot(count, wins, 'b-', label='P1 Win')
plt.plot(count, losses, 'r-', label='P2 Win')
plt.plot(count, ties, 'g-', label='Tie')
plt.legend(loc='best')
plt.show()

In [ ]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

In [ ]:
!apt-get install python3-tk
!apt-get install -y xvfb # Install X Virtual Frame Buffer
import os
os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
os.environ['DISPLAY']=':1.0'

In [ ]:
import numpy as np

class Trainer:

    def __init__(self, agent, learning_parameter = 0.1, discount_factor = 0.9, Q = {}):
        """
            agent (Agent)
            learning_parameter (float)
            discount_factor (float)
            Q (dict)
        """

        self.agent = agent
        self.learning_parameter = learning_parameter
        self.discount_factor = discount_factor
        self.Q = Q

    def getStatePairKey(state_hash, action_hash):
        """ Returns state-pair hash key, requires separate state and action hash keys first """
        return state_hash*action_hash

    def getValueQ(self, state_hash, action_hash):
        """ Get expected reward given an action in a given state,
            returns 0 if the state-action pair has not been seen before.
            Input is state and action hash key                          """

        state_action_key = Trainer.getStatePairKey(state_hash, action_hash)
        if state_action_key in self.Q:
            return self.Q.get(state_action_key)
        else:
            self.Q[state_action_key] = 0
            return 0

    def setValueQ(self, state_hash, action_hash, value):
        """ Set value in Q """
        state_action_key = Trainer.getStatePairKey(state_hash, action_hash)

        self.Q[state_action_key] = value

    def getMaxQ(self, state_hash, list_action_hash):
        """ Returns the maximum Q value given a state and list of actions (input is hash keys) """
        maxQ = 0
        for a_hash in list_action_hash:
            tmpQ = self.getValueQ(state_hash, a_hash)
            if maxQ < tmpQ:
                maxQ = tmpQ
        return maxQ

    def getBestAction(self, state_hash, list_action_hash, list_actions):
        """ Get best action given a set of possible actions in a given state """

        # Pick a random action at first
        random_idx = np.random.choice(list_actions.shape[0])
        best_action = list_actions[random_idx]

        # Find action that given largest Q in given state
        maxQ = 0
        for a_hash, action in zip(list_action_hash, list_actions):
            tmpQ = self.getValueQ(state_hash, a_hash)
            if maxQ < tmpQ:
                maxQ = tmpQ
                best_action = action

        return best_action


    def updateQ(self, state, action):
        """ Implements Q-learning iterative algorithm """


        state_hash = state.getStateHash()
        action_hash = self.agent.getActionHash(action)

        # Get current Q Value
        currentQ = self.getValueQ(state_hash, action_hash)

        # Find max Q value given the possible set of actions in the next state
        next_state, next_actions = self.agent.getActionHashFromState(action=action, state=state)
        max_nextQ = self.getMaxQ(next_state, next_actions)

        # Update new Q
        newQ =  (1-self.learning_parameter)*currentQ
        newQ += self.learning_parameter*(self.agent.rewardFunction(state, action) + self.discount_factor*max_nextQ)

        self.setValueQ(state_hash, action_hash, newQ)

In [ ]:
import numpy as np

class Board:
    """ Class that represents the game board of Tic Tac Toe """

    playerX = 1
    playerO = -1

    def __init__(self, rows = 3, cols = 3, win_threshold = 3):
        """
            rows (int)
            cols (int)
            win_threshold (int) - Do not change
        """
        self.state = np.zeros((rows, cols), dtype=np.int8)
        self.rows = rows
        self.cols = cols
        self.win_threshold = win_threshold

    def getState(self):
        """ Get state of game """
        return self.state

    def getPosition(self, x, y):
        """ Get state at position (x,y) """
        return self.state[x,y]

    def setPosition(self, x, y, value):
        """  Set state at position (x,y) with value """
        self.state[x,y] = value

    def getAvailablePos(self):
        """  Get state positions that have no value (non-zero) """
        return np.argwhere(self.state == 0)

    def getStateHash(self, inverted=False):
        """  Get hash key of state """
        factor = 1
        state_hash = 0
        for i in range(self.rows):
            for j in range(self.cols):

                if inverted:
                    state_hash -= self.state[i,j]*factor
                else:
                    state_hash += self.state[i,j]*factor

                factor = 10*factor
        return state_hash

    def checkWinner(self):
        """  Get winner, if one exists """
        """ TODO: Not general case, only works for 3x3 board """

        symbols = np.unique(self.state)
        symbols = list(symbols[np.nonzero(symbols)])

        for sym in symbols:

            # Check rows
            row= np.any((np.all(self.state == sym, axis=1)))

            # Check columns
            col = np.any((np.all(self.state == sym, axis=0)))

            # Check diagonals
            diag1 = np.array([self.state[0,0], self.state[1,1], self.state[2,2]])
            diag1 = np.all(diag1 == sym)

            diag2 = np.array([self.state[0,2], self.state[1,1], self.state[2,0]])
            diag2 = np.all(diag2 == sym)

            # Check if state has winner and return winner in that case
            if row or col or diag1 or diag2:
                return sym

        # No winner found
        return 0

    def checkGameEnded(self):
        """ Check if game has ended by observing if there any possible moves left """
        return len(self.getAvailablePos()) == 0

    def checkWinPossible(self, last_player_value):
        """
            Test whether there is a winning move available for the next player.
            Return True if it is available.
            last_player (int)
        """

        # Next player is the negative of last_player_value
        next_player_player = - last_player_value

        winning_move_found = False
        for action in self.getAvailablePos():
            x = action[0]
            y = action[1]

            # Perform action
            self.setPosition(x, y, next_player_player)

            # Check if winning move
            if self.checkWinner() != 0:
                winning_move_found = True

            # Revert action
            self.setPosition(x, y, 0)

            # If found, then return True
            if winning_move_found is True:
                return True

        return False



    def resetGame(self):
        """ Reset game """
        self.state = np.zeros((self.rows, self.cols), dtype=np.int16)

    def getInvertedState(self):
        """ Return state where player O and X have swapped places """
        return -self.state

    def __str__(self):
        return str(self.state)

In [ ]:
import numpy as np
import _pickle as cPickle

class Agent:
    """  Class that defines agent and its possible actions """


    def __init__(self, symbol, state, load_trainer = None):
        """
        symbol (string)
        state (Board)
        load_trainer (string) - Path to saved trainer
        """

        self.symbol = symbol
        self.current_state = state

        self.actions = self.current_state.getAvailablePos()
        self.action_history = []

        if load_trainer is None:
            self.trainer = Trainer(self)
        else:
            self.trainer = self.loadTrainer(load_trainer)

    def getPossibleActions(self):
        """ Get possible actions """
        self.updatePossibleActions()
        return self.actions

    def updatePossibleActions(self):
        """ Update possible actions """
        self.actions = self.current_state.getAvailablePos()

    def performAction(self, action, state = None, updateQ = False):
        """
            Make move from agent, updates the state and possible actions.
            Also updates Q at the same time.
        """

        assert action.shape == (2,), "Wrong shape " + str(action)

        if state == None:
            state = self.current_state

        # Read action
        x = action[0]
        y = action[1]

        # Update Q as part of Q-learning in the Trainer class
        if updateQ is True:
            self.trainer.updateQ(state, action)

        # Make move
        state.setPosition(x, y, self.symbol)
        self.action_history.append(action)

        # Update possible actions
        self.updatePossibleActions()

    def performRandomAction(self, updateQ=True):
        """ Perform random actions, important for exploration of state-pairs """

        self.updatePossibleActions()
        random_idx = np.random.choice(self.actions.shape[0])
        action = self.actions[random_idx]

        self.performAction(action, updateQ=updateQ)

        return action

    def revertLastAction(self, state = None):
        """ Make move from agent, updates the state and possible actions  """

        if state == None:
            state = self.current_state

        # Get last action
        last_action = self.action_history.pop()
        x = last_action[0]
        y = last_action[1]

        # Set to zero
        state.setPosition(x, y, 0)

        # Update possible actions
        self.updatePossibleActions()

    def getActionHash(self, action):
        """ Get hash key of action """
        action_hash = 10*(action[0]+1) + (action[1]+1)
        return action_hash

    def getActionHashFromState(self, action = None, state = None):
        """ Get hash key of actions in a given state, also returns the hash key of that state """

        if state is None:
            state = self.current_state

        if not action is None:
            self.performAction(action, state=state)

        next_state_hash = state.getStateHash()
        next_actions_hash = []
        for a in self.actions:
            next_actions_hash.append(self.getActionHash(a))

        if not action is None:
            self.revertLastAction(state=state)

        return next_state_hash, next_actions_hash

    def rewardFunction(self, state, action):
        """ Returns positive value actions turns into win, else zero """

        # Perform action
        self.performAction(action, state=state)

        # Check winner
        winner = state.checkWinner()
        missed_blocking_move = state.checkWinPossible(self.symbol)
        if winner == self.symbol:
                reward = 1
        elif missed_blocking_move is True:
                reward = -1
        else:
            reward = 0
        # Revert action
        self.revertLastAction(state=state)

        return reward

    def assignState(self, state):
        """ Assign a state (Board) to the agent"""
        self.current_state = state
        self.updatePossibleActions()

    def getBestAction(self):
        """ Get best move from the Trainer that has the largest expected reward """

        self.updatePossibleActions()

        # Get hash key for state and actions
        state_hash, actions_hash = self.getActionHashFromState()

        # Return best move (if all are equally good, then it picks one at random)
        return self.trainer.getBestAction(state_hash, actions_hash, self.actions)


    def saveTrainer(self, save_path):
        """ Saves agent to save_path (str) """
        dict = self.trainer.Q
        with open(save_path, "wb") as f:
            cPickle.dump(dict, f)

    def loadTrainer(self, save_path):
        """
        Load Q-values from another trainer
        """

        with open(save_path, "rb") as f:
            dict = cPickle.load(f)

        return Trainer(self, Q=dict)

In [ ]:

from tqdm import tqdm
import random

player1_symbol = Board.playerX
player2_symbol = Board.playerO

def simulate(iterations, explore_only = False, save_agent1 = None):
    """
        iterations (int)
        explore_only (bool) - If true, then only explore.
                              Else,  follow an epsilon-greedy policy that lowers the probability to explore over time.
    """

    # Construct game board
    game = Board()
    exploration_probability = 1

    # Initialize players
    agent1 = Agent(player1_symbol, game)
    agent2 = Agent(player2_symbol, game)

    # Counters for wins of each agent and total number of games
    nbr_wins_agent1 = 0
    nbr_wins_agent2 = 0
    nbr_games = 0

    # Pick current player
    current_player = player1_symbol

    # Epsilon-greedy
    exploration_probability = 1.0

    # Start iterations
    for i in tqdm(range(iterations)):

        # Check if games has ended, reset if True
        if game.checkGameEnded():
            nbr_games += 1
            game.resetGame()
            agent1.updatePossibleActions()
            agent2.updatePossibleActions()

        # Check who is the current player
        if current_player == agent1.symbol:
            a = agent1
        else:
            a = agent2

        # Explore
        if explore_only is True or random.random() < exploration_probability:
            a.performRandomAction(updateQ=True)
        # Exploit
        else:
            best_action = a.getBestAction()
            a.performAction(best_action, updateQ=(True))

        # Reduce probability to explore during training
        # Do not remove completely
        if exploration_probability > 0.2:
            exploration_probability -= 1/iterations

        # Check if there is a winner
        winner = game.checkWinner() # Returns 0 if there is no winner
        if winner != 0:

            # Reset game and retrieve
            nbr_games += 1
            game.resetGame()

            # Add to count for corresponding winner
            if winner == agent1.symbol:
                nbr_wins_agent1 += 1
            else:
                nbr_wins_agent2 += 1

        # Swap player
        if current_player == player1_symbol:
            current_player = player2_symbol
        else:
            current_player = player1_symbol


    # Print outcome
    print(nbr_wins_agent1, nbr_wins_agent2, nbr_games)
    print("Win percentage: Agent 1 {:.2%}, Agent 2 {:.2%}.".format(nbr_wins_agent1/nbr_games, nbr_wins_agent2/nbr_games))

    if save_agent1 is not None:
        print("Saved trainer of agent 1 to {}".format(save_agent1))
        agent1.saveTrainer(save_agent1)

    # Return agents
    return agent1, agent2

if __name__ == "__main__":

    # Train two agents against each other and save it in playerX.pkl
    iterations = 250000
    agent1, agent2 = simulate(iterations, explore_only=False, save_agent1="playerX.pkl")

In [ ]:
import tkinter as tk
import random
import time
import numpy as np

import matplotlib
import os


playerX = Board.playerX
playerO = Board.playerO
agent_save = "playerX.pkl"

class Application(tk.Frame):
    def __init__(self, master=None):
        if os.environ.get('DISPLAY','') == '':
            print('Currently no display found. Using the non-interactive Agg backend')
            matplotlib.use('Agg')
        tk.Frame.__init__(self, master)
        self.master.title("Tic-tac-toe")
        self.master.minsize(300, 300)

        # Start game
        self.board = Board(rows=3, cols=3, win_threshold=3)
        self.current_player = playerX if (random.random() < 0.5) else playerO

        # Train agent and assign it to this game
        print("Preparing agent")
        self.agent = Agent(playerX, self.board, load_trainer=agent_save)
        self.agent_symbol = self.agent.symbol

        # Create widgets
        self.createBoard(self.board)

        if self.current_player == self.agent_symbol:
            self.agentMove()

        self.pack(fill="both")

    def createBoard(self, board):

        # Text label
        self.info_label = tk.Label(self, text = "Player {}'s turn".format("X" if self.current_player == playerX else "O") )


        # Create buttons
        self.button00 = tk.Button(self, height = 4, width = 8, text=' ', font='Times 20 bold', bg='gray', fg='white', command = lambda : self.playMove(0,0))
        self.button01 = tk.Button(self, height = 4, width = 8, text=' ', font='Times 20 bold', bg='gray', fg='white', command = lambda : self.playMove(0,1))
        self.button02 = tk.Button(self, height = 4, width = 8, text=' ', font='Times 20 bold', bg='gray', fg='white', command = lambda : self.playMove(0,2))

        self.button10 = tk.Button(self, height = 4, width = 8, text=' ', font='Times 20 bold', bg='gray', fg='white', command = lambda : self.playMove(1,0))
        self.button11 = tk.Button(self, height = 4, width = 8, text=' ', font='Times 20 bold', bg='gray', fg='white', command = lambda : self.playMove(1,1))
        self.button12 = tk.Button(self, height = 4, width = 8, text=' ', font='Times 20 bold', bg='gray', fg='white', command = lambda : self.playMove(1,2))

        self.button20 = tk.Button(self, height = 4, width = 8, text=' ', font='Times 20 bold', bg='gray', fg='white', command = lambda : self.playMove(2,0))
        self.button21 = tk.Button(self, height = 4, width = 8, text=' ', font='Times 20 bold', bg='gray', fg='white', command = lambda : self.playMove(2,1))
        self.button22 = tk.Button(self, height = 4, width = 8, text=' ', font='Times 20 bold', bg='gray', fg='white', command = lambda : self.playMove(2,2))

        self.reset_button = tk.Button(self, text = "Reset", command = self.resetGame)
        self.Q_button = tk.Button(self, text="Display Q values", command = self.displayQ, relief='raised')

        # Insert to grid
        self.info_label.grid(row = 0, column = 1)

        self.button00.grid(row = 1, column = 0)
        self.button01.grid(row = 1, column = 1)
        self.button02.grid(row = 1, column = 2)

        self.button10.grid(row = 2, column = 0)
        self.button11.grid(row = 2, column = 1)
        self.button12.grid(row = 2, column = 2)

        self.button20.grid(row = 3, column = 0)
        self.button21.grid(row = 3, column = 1)
        self.button22.grid(row = 3, column = 2)

        self.reset_button.grid(row = 4, column = 1)
        self.Q_button.grid(row = 4, column = 2)


    def getText(self, x, y):
        value = self.board.getPosition(x, y)
        if value == playerX:
            return "X"
        elif value == playerO:
            return "O"
        else:
            return " "

    def getQValue(self, x, y):
        value = self.getText(x,y)
        if value == " ":
            action_hash = self.agent.getActionHash(np.asarray([x,y]))
            state_hash = self.board.getStateHash(inverted=True)
            Q = self.agent.trainer.getValueQ(state_hash, action_hash)
            return "{:.2f}".format(Q)
        else:
            return value

    def updateText(self):

        self.info_label.configure(text="Player {}'s turn".format("X" if self.current_player == playerX else "O"))

        self.button00.configure(text=self.getText(0,0), state=tk.ACTIVE)
        self.button01.configure(text=self.getText(0,1), state=tk.ACTIVE)
        self.button02.configure(text=self.getText(0,2), state=tk.ACTIVE)
        self.button10.configure(text=self.getText(1,0), state=tk.ACTIVE)
        self.button11.configure(text=self.getText(1,1), state=tk.ACTIVE)
        self.button12.configure(text=self.getText(1,2), state=tk.ACTIVE)
        self.button20.configure(text=self.getText(2,0), state=tk.ACTIVE)
        self.button21.configure(text=self.getText(2,1), state=tk.ACTIVE)
        self.button22.configure(text=self.getText(2,2), state=tk.ACTIVE)

        self.reset_button.configure(state=tk.ACTIVE)

    def displayQ(self):

        if self.Q_button.config('relief')[-1] == 'sunken':
            self.updateText()
            self.Q_button.configure(text="Display Q values", relief='raised')
        else:
            self.button00.configure(text=self.getQValue(0,0), state=tk.DISABLED)
            self.button01.configure(text=self.getQValue(0,1), state=tk.DISABLED)
            self.button02.configure(text=self.getQValue(0,2), state=tk.DISABLED)
            self.button10.configure(text=self.getQValue(1,0), state=tk.DISABLED)
            self.button11.configure(text=self.getQValue(1,1), state=tk.DISABLED)
            self.button12.configure(text=self.getQValue(1,2), state=tk.DISABLED)
            self.button20.configure(text=self.getQValue(2,0), state=tk.DISABLED)
            self.button21.configure(text=self.getQValue(2,1), state=tk.DISABLED)
            self.button22.configure(text=self.getQValue(2,2), state=tk.DISABLED)
            self.reset_button.configure(state=tk.DISABLED)

            self.Q_button.configure(text="Hide Q values", relief='sunken')




    def playMove(self, x, y):

        # Raise error because position is non-empty
        if self.board.getPosition(x,y) == playerX or self.board.getPosition(x,y) == playerO:
            print("Error: This action is not possible, pick another position.")
        else:
            # Update board
            self.board.setPosition(x, y, self.current_player)

            # Swap player
            if self.current_player == playerX:
                self.current_player = playerO
            else:
                self.current_player = playerX

            # Update text
            self.updateText()

            # Check if game has ended
            winner = self.board.checkWinner()
            if winner != 0 or self.board.checkGameEnded():
                self.endGame()

            elif self.current_player == self.agent_symbol:
                self.agentMove()

    def agentMove(self):
        move = self.agent.getBestAction()
        time.sleep(random.random()*1 + 0.5)
        self.playMove(move[0], move[1])

    def endGame(self):
        winner = self.board.checkWinner()
        if winner == playerX:
            self.info_label.configure(text = "Player X won!")
        elif winner == playerO:
            self.info_label.configure(text = "Player O won!")
        else:
            self.info_label.configure(text = "Game tied")

    def resetGame(self):
        self.board.resetGame()
        self.current_player = playerX if (random.random() < 0.5) else playerO
        if self.current_player == self.agent_symbol:
            self.agentMove()
        self.updateText()



if __name__ == "__main__":
    Application()
    tk.mainloop()
    root.mainloop()

In [ ]:
tf.compat.v1.enable_v2_behavior()

# Set up a virtual display for rendering OpenAI gym environments.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

In [ ]:
tf.version.VERSION

## Hyperparameters

In [ ]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

## Environment

In Reinforcement Learning (RL), an environment represents the task or problem to be solved. Standard environments can be created in TF-Agents using `tf_agents.environments` suites. TF-Agents has suites for loading environments from sources such as the OpenAI Gym, Atari, and DM Control.

Load the CartPole environment from the OpenAI Gym suite.

In [ ]:
env_name = 'CartPole-v0'
env = suite_gym.load(env_name)

You can render this environment to see how it looks. A free-swinging pole is attached to a cart.  The goal is to move the cart right or left in order to keep the pole pointing up.

In [ ]:
#@test {"skip": true}
env.reset()
PIL.Image.fromarray(env.render())

The `environment.step` method takes an `action` in the environment and returns a `TimeStep` tuple containing the next observation of the environment and the reward for the action.

The `time_step_spec()` method returns the specification for the `TimeStep` tuple. Its `observation` attribute shows the shape of observations, the data types, and the ranges of allowed values. The `reward` attribute shows the same details for the reward.


In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)

In [ ]:
print('Reward Spec:')
print(env.time_step_spec().reward)

The `action_spec()` method returns the shape, data types, and allowed values of valid actions.

In [ ]:
print('Action Spec:')
print(env.action_spec())

In the Cartpole environment:

-   `observation` is an array of 4 floats:
    -   the position and velocity of the cart
    -   the angular position and velocity of the pole
-   `reward` is a scalar float value
-   `action` is a scalar integer with only two possible values:
    -   `0` — "move left"
    -   `1` — "move right"


In [ ]:
time_step = env.reset()
print('Time step:')
print(time_step)

action = np.array(1, dtype=np.int32)

next_time_step = env.step(action)
print('Next time step:')
print(next_time_step)

Usually two environments are instantiated: one for training and one for evaluation.

In [ ]:
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

The Cartpole environment, like most environments, is written in pure Python. This is converted to TensorFlow using the `TFPyEnvironment` wrapper.

The original environment's API uses Numpy arrays. The `TFPyEnvironment` converts these to `Tensors` to make it compatible with Tensorflow agents and policies.


In [ ]:
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

## Agent

The algorithm used to solve an RL problem is represented by an `Agent`. TF-Agents provides standard implementations of a variety of `Agents`, including:

-   [DQN](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) (used in this tutorial)
-   [REINFORCE](http://www-anw.cs.umass.edu/~barto/courses/cs687/williams92simple.pdf)
-   [DDPG](https://arxiv.org/pdf/1509.02971.pdf)
-   [TD3](https://arxiv.org/pdf/1802.09477.pdf)
-   [PPO](https://arxiv.org/abs/1707.06347)
-   [SAC](https://arxiv.org/abs/1801.01290).

The DQN agent can be used in any environment which has a discrete action space.

At the heart of a DQN Agent is a `QNetwork`, a neural network model that can learn to predict `QValues` (expected returns) for all actions, given an observation from the environment.

Use `tf_agents.networks.q_network` to create a `QNetwork`, passing in the `observation_spec`, `action_spec`, and a tuple describing the number and size of the model's hidden layers.


In [ ]:
fc_layer_params = (100,)

q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)

Now use `tf_agents.agents.dqn.dqn_agent` to instantiate a `DqnAgent`. In addition to the `time_step_spec`, `action_spec` and the QNetwork, the agent constructor also requires an optimizer (in this case, `AdamOptimizer`), a loss function, and an integer step counter.

In [ ]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

## Policies

A policy defines the way an agent acts in an environment. Typically, the goal of reinforcement learning is to train the underlying model until the policy produces the desired outcome.

In this tutorial:

-   The desired outcome is keeping the pole balanced upright over the cart.
-   The policy returns an action (left or right) for each `time_step` observation.

Agents contain two policies:

-   `agent.policy` — The main policy that is used for evaluation and deployment.
-   `agent.collect_policy` — A second policy that is used for data collection.


In [ ]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

Policies can be created independently of agents. For example, use `tf_agents.policies.random_tf_policy` to create a policy which will randomly select an action for each `time_step`.

In [ ]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

To get an action from a policy, call the `policy.action(time_step)` method. The `time_step` contains the observation from the environment. This method returns a `PolicyStep`, which is a named tuple with three components:

-   `action` — the action to be taken (in this case, `0` or `1`)
-   `state` — used for stateful (that is, RNN-based) policies
-   `info` — auxiliary data, such as log probabilities of actions

In [ ]:
example_environment = tf_py_environment.TFPyEnvironment(
    suite_gym.load('CartPole-v0'))

In [ ]:
time_step = example_environment.reset()

In [ ]:
random_policy.action(time_step)

## Metrics and Evaluation

The most common metric used to evaluate a policy is the average return. The return is the sum of rewards obtained while running a policy in an environment for an episode. Several episodes are run, creating an average return.

The following function computes the average return of a policy, given the policy, environment, and a number of episodes.


In [ ]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

Running this computation on the `random_policy` shows a baseline performance in the environment.

In [ ]:
compute_avg_return(eval_env, random_policy, num_eval_episodes)

## Replay Buffer

The replay buffer keeps track of data collected from the environment. This tutorial uses `tf_agents.replay_buffers.tf_uniform_replay_buffer.TFUniformReplayBuffer`, as it is the most common.

The constructor requires the specs for the data it will be collecting. This is available from the agent using the `collect_data_spec` method. The batch size and maximum buffer length are also required.


In [ ]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

For most agents, `collect_data_spec` is a named tuple called `Trajectory`, containing the specs for observations, actions, rewards, and other items.

In [ ]:
agent.collect_data_spec

In [ ]:
agent.collect_data_spec._fields

## Data Collection

Now execute the random policy in the environment for a few steps, recording the data in the replay buffer.

In [ ]:
#@test {"skip": true}
def collect_step(environment, policy, buffer):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)

  # Add trajectory to the replay buffer
  buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps):
  for _ in range(steps):
    collect_step(env, policy, buffer)

collect_data(train_env, random_policy, replay_buffer, initial_collect_steps)

# This loop is so common in RL, that we provide standard implementations.
# For more details see the drivers module.
# https://www.tensorflow.org/agents/api_docs/python/tf_agents/drivers

The replay buffer is now a collection of Trajectories.

In [ ]:
# For the curious:
# Uncomment to peel one of these off and inspect it.
# iter(replay_buffer.as_dataset()).next()

The agent needs access to the replay buffer. This is provided by creating an iterable `tf.data.Dataset` pipeline which will feed data to the agent.

Each row of the replay buffer only stores a single observation step. But since the DQN Agent needs both the current and next observation to compute the loss, the dataset pipeline will sample two adjacent rows for each item in the batch (`num_steps=2`).

This dataset is also optimized by running parallel calls and prefetching data.

In [ ]:
# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2).prefetch(3)


dataset

In [ ]:
iterator = iter(dataset)

print(iterator)


In [ ]:
# For the curious:
# Uncomment to see what the dataset iterator is feeding to the agent.
# Compare this representation of replay data
# to the collection of individual trajectories shown earlier.

# iterator.next()

## Training the agent

Two things must happen during the training loop:

-   collect data from the environment
-   use that data to train the agent's neural network(s)

This example also periodicially evaluates the policy and prints the current score.

The following will take ~5 minutes to run.

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):

  # Collect a few steps using collect_policy and save to the replay buffer.
  collect_data(train_env, agent.collect_policy, replay_buffer, collect_steps_per_iteration)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

## Visualization


### Plots

Use `matplotlib.pyplot` to chart how the policy improved during training.

One iteration of `Cartpole-v0` consists of 200 time steps. The environment gives a reward of `+1` for each step the pole stays up, so the maximum return for one episode is 200. The charts shows the return increasing towards that maximum each time it is evaluated during training. (It may be a little unstable and not increase monotonically each time.)

In [ ]:
#@test {"skip": true}

iterations = range(0, num_iterations + 1, eval_interval)
plt.plot(iterations, returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')
plt.ylim(top=250)

### Videos

Charts are nice. But more exciting is seeing an agent actually performing a task in an environment.

First, create a function to embed videos in the notebook.

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

Now iterate through a few episodes of the Cartpole game with the agent. The underlying Python environment (the one "inside" the TensorFlow environment wrapper) provides a `render()` method, which outputs an image of the environment state. These can be collected into a video.

In [ ]:
def create_policy_eval_video(policy, filename, num_episodes=5, fps=30):
  filename = filename + ".mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    for _ in range(num_episodes):
      time_step = eval_env.reset()
      video.append_data(eval_py_env.render())
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = eval_env.step(action_step.action)
        video.append_data(eval_py_env.render())
  return embed_mp4(filename)




create_policy_eval_video(agent.policy, "trained-agent")

For fun, compare the trained agent (above) to an agent moving randomly. (It does not do as well.)

In [ ]:
create_policy_eval_video(random_policy, "random-agent")